In [476]:
import random
random.seed(42) # define the seed (important to reproduce the results)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [477]:
data = pd.read_csv('../dataset/breast-cancer.csv', sep = ';')
data = data.dropna(axis='rows') #remove NaN


In [478]:
print("Número de linhas e colunas na matriz de atributos:", data.shape)
attributes = list(data.columns)
data.head(10) # showing the first ten lines 

Número de linhas e colunas na matriz de atributos: (286, 10)


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,03/mai,yes,2,left,right_up,no,recurrence-events
5,50-59,premeno,25-29,03/mai,no,2,right,left_up,yes,no-recurrence-events
6,50-59,ge40,40-44,0-2,no,3,left,left_up,no,no-recurrence-events
7,40-49,premeno,out/14,0-2,no,2,left,left_up,no,no-recurrence-events
8,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
9,40-49,ge40,40-44,15-17,yes,2,right,left_up,yes,no-recurrence-events


Vamos remover as linhas que contenham "?" ou qualquer outro caracter que indique um erro ou entrada faltante.

Primeiramente, convertemos essas entradas para "NaN":

In [479]:
import numpy as np
# Substitui Nan por um caracter desejado
data = data.replace('?', np.nan) # presente em dois atributos da base 'breast-cancer'

Apagando as linhas com NaN

In [480]:
# remove as linhas com NaN
data = data.dropna() 

## Binarização dos dados

In [481]:
x_prev = data.iloc[:, 0:9].values

In [482]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder_NodeCaps = LabelEncoder()
label_encoder_Breast = LabelEncoder()
label_encoder_Irradiat = LabelEncoder()

In [483]:
x_prev[:,4] = label_encoder_NodeCaps.fit_transform(x_prev[:,4])
x_prev[:,6] = label_encoder_Breast.fit_transform(x_prev[:,6])
x_prev[:,8] = label_encoder_Irradiat.fit_transform(x_prev[:,8])

| OnehotEncoder - Converter valores nominais e ordinais para inteiro 

**Contando quantas opções de resposta tem um determinado atributo**

In [484]:
len(np.unique(data['node-caps'])) # atributo inicialmente com um valor '?'

2

In [485]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [486]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,1,2,3,5,7])])

In [487]:
x_prev = onehotencoder.fit_transform(x_prev)

In [488]:
x_prev.shape

(277, 35)

In [489]:
nrow,ncol = x_prev.shape
# armazeno os valores do atributo alvo (class)
y_class = data.iloc[:, 9].values 

Selecionando os conjuntos de treinamento e teste.

In [490]:
from sklearn.model_selection import train_test_split
p = 0.7 # fracao de elementos no conjunto de treinamento
X_train, X_test, y_train, y_test = train_test_split(x_prev, y_class, train_size = p, random_state = 42)

Classificação usando a biblioteca scikit-learn

In [491]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.